# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment
from azureml.core import ScriptRunConfig
import os

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'titanic-survival'
project_folder = './capstone-project'

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')
experiment=Experiment(ws, experiment_name)

key = "titanic-survival-data"
description_text = "Kaggle dataset for Titanic disaster."

if key in ws.datasets.keys(): 
        dataset = ws.datasets[key] 

else:
        # Create AutoML Dataset and register it into Workspace
        my_dataset = 'https://raw.githubusercontent.com/j0h4nnesk/Capstone_project_Titanic_Survival/main/train.csv'
        dataset = Dataset.Tabular.from_delimited_files(my_dataset)        
        # Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


run = experiment.start_logging()

capstone-project
capstone-project
francecentral
2d4b3a3e-de2a-45bb-9ac0-29caf8f98da4


In [3]:
# Name for the cluster
cpu_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    # Poll for a minimum number of nodes (min_nodes = 1). 
    # If no min node count is provided it uses the scale settings for the cluster.
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2', min_nodes=1, max_nodes=6)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 2, 'targetNodeCount': 2, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 2, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-03-03T14:06:36.844000+00:00', 'errors': [{'error': {'code': 'ClusterCoreQuotaReached', 'message': 'Operation results in exceeding quota limits of Total Cluster Dedicated Regional vCPUs. Maximum allowed: 10, Current in use: 10, Additional requested: 8. Please contact support to increase the quota for resource type Total Cluster Dedicated Regional vCPUs'}}], 'creationTime': '2021-03-03T13:45:29.749653+00:00', 'modifiedTime': '2021-03-03T13:45:45.370277+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCou

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [4]:
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='./conda_dependencies.yml')
compute_target = ws.compute_targets['compute-cluster']

# Specify a Policy 
policy = BanditPolicy(evaluation_interval=2,slack_factor=0.1,delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")
script_folder = './training'
os.makedirs(script_folder, exist_ok=True)
import shutil
shutil.copy('./train.py', script_folder)

src = ScriptRunConfig(source_directory=script_folder,
                        script='train.py',
                        #arguments=['--input_data', dataset.as_named_input('titanic')],
                        compute_target=compute_target,
                        environment=sklearn_env)


# Specify parameter sampler
ps = RandomParameterSampling({
    '--max_iter' : choice(20,40,80,100,150,200),
    '--C' : choice(0.001,0.01,0.1, 0.5, 1,1.5,10,20,50,100)
}) 


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling = ps, 
                                    primary_metric_name = 'Accuracy',
                                    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                    policy=policy,
                                    run_config=src,
                                    max_total_runs=20,
                                    max_concurrent_runs=4)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
# Submit hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = experiment.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status()=='Completed')

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_abe40fc9-4e9b-4147-9678-54405329a6da
Web View: https://ml.azure.com/experiments/titanic-survival/runs/HD_abe40fc9-4e9b-4147-9678-54405329a6da?wsid=/subscriptions/2d4b3a3e-de2a-45bb-9ac0-29caf8f98da4/resourcegroups/capstone-project/workspaces/capstone-project

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-03-03T15:25:01.284245][API][INFO]Experiment created<END>\n""<START>[2021-03-03T15:25:01.876537][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2021-03-03T15:25:02.0381969Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-03-03T15:25:02.045743][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run

In [ ]:
# Get your best run and save the model from that run.
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters

# Identify the best performing configuration and save the model from that run. 

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print(parameter_values[0],":", parameter_values[1],"and",parameter_values[2],":",parameter_values[3])
print('Accuracy:', best_run_metrics['Accuracy'])

In [ ]:
# get_metrics(): returns the metrics from all the runs that were launched by this HyperDriveRun.
print("Best run metrics :",best_run.get_metrics())
print('***************************************************')

# get_details(): returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())
print('***************************************************')

# get_file_names(): returns a list of the files that are stored in association with the run.
print("Best run file names :",best_run.get_file_names())
print('***************************************************')

In [ ]:
# Save the best model

best_run.register_model(model_name = "best_run_hyperdrive.pkl", model_path = './outputs/')
print(best_run)

In [ ]:
# Download the model file
best_run.download_file('outputs/model.pkl', 'hyperdrive_model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service